In [3]:
import numpy as np 
psi=np.zeros((2,2,2,2)) 
psi[0,0,0,0]=psi[1,1,1,1]=1/np.sqrt(2) 
H_matrix=1/np.sqrt(2)*np.array([[1, 1],
                                [1,-1]])
psi_prime=np.tensordot(H_matrix,psi,(1,0)) 
psi_prime=np.tensordot(H_matrix,psi,(1,1)) 
psi_prime=np.moveaxis(psi_prime,0,1)
class Reg: 
    def __init__(self,n):
        self.n=n
        self.psi=np.zeros((2,)*n) 
        self.psi[(0,)*n]=1 
def H(i,reg): 
    reg.psi=np.tensordot(H_matrix,reg.psi,(1,i)) 
    reg.psi=np.moveaxis(reg.psi,0,i)
CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])

CNOT_tensor=np.reshape(CNOT_matrix, (2,2,2,2))
def CNOT(control, target, reg):
    reg.psi=np.tensordot(CNOT_tensor, reg.psi, ((2,3),(control, target))) 
    reg.psi=np.moveaxis(reg.psi,(0,1),(control,target))
def generate_GHZ(reg): 
    H(0,reg)
    for i in range(reg.n-1):
        CNOT(i,i+1,reg)
projectors=[np.array([[1,0],[0,0]]), np.array([[0,0],[0,1]]) ] 
def project(i,j,reg): 
    projected=np.tensordot(projectors[j],reg.psi,(1,i))
    return np.moveaxis(projected,0,i)

from scipy.linalg import norm 

def measure(i,reg): 
    projected=project(i,0,reg) 
    norm_projected=norm(projected.flatten()) 
    if np.random.random()<norm_projected**2: 
        reg.psi=projected/norm_projected
        return 0
    else:
        projected=project(i,1,reg)
        reg.psi=projected/norm(projected)
        return 1
list3=[]
for i in range(5000):
    reg=Reg(4)
    H(0,reg)
    print(measure(0,reg), end='')
    list3.append(measure(0,reg))
x=list3.count("0")
print(x, len(list3))
print([*list3])

0010111100011111100011110110000101010101100000010111000011001000100001011101101100001010010011010101100000100010011101111111011100010100001001101000101111010001101100110100001111100101010001111011011111011011110011101001011100010000100001101111100000100101000100011000111001010000001010010001111000111101000100000111100011101111100100011001110101000010101001101111101101001110011101010001011110001110011001011010010100101111101000011010111111110101110001001000101111110000001011111011100110000100000000101011111100100001100111101111010001010100000111111001001000100001111011000001001110001101100101100111100110011010011000001000011011111011110011010000111010111110001011111100001001000100101111100110010110111110110011111111001000001001111001011100100000101100101110000000010010010101110011111111010111101101111101010011001011100001001001101100100011101100010101001101101011100111110011111000001111011110001110111011110100000000010101101101100101010011010101100111110011111111001010111110000101110111